In [ ]:
import gym
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import deque

In [ ]:
# Create the environment 
environment_name = 'CarRacing-v0'
environment = gym.make(environment_name)

In [ ]:
class NN_Model(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=48, kernel_size=8, stride=1, padding=0, bias=False)
        self.conv2 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=4, stride=1, padding=0, bias=False)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        self.fc1 = nn.Linear(100,128)
        self.fc2= nn.Linear(128, 15)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
  
    

In [ ]:
class DQNAgent():
    def __init__(self):

        self.action_space = [
            (-1, 1, 0), (-1, 1, 0.4), (-1, 0, 0), (-1, 0, 0.4), (-1, 0, 1), # Action Space: (Steer, Accelerate, Brake)
            (1,  1, 0), (1, 1,  0.4), (1,  0, 0), (1,  0, 0.4), (1,  0, 1),  # Steer left = -1, Steer right = 1 
            (0,  1, 0), (0, 1,  0.4), (0,  0, 1), (0,  0, 0.4), (0,  0, 0)   # Accelerate = Between 0 - 1      
        ]                                                                   # Break = Between 0 - 1
        
        self.gamma = 0.95 # 0 means prioritise immediate rewards, 1 means prioritise future rewards
        self.epsilon = 1 # Exploration rate 
        self.epsilon_decay = 0.9995 # The rate of decreasing the exploration rate
        self.epsilon_min = 0.01 # The minimum exploration rate 
        self.learning_rate = 0.001 
        self.memory = deque(maxlen=3000) # Basically an array but last element gets removed when a new one is added
        self.batch_size = 64
        
        self.optimizer = optim.Adam(model.parameters(), self.learning_rate)
        self.mse_loss = nn.MSELoss() # Mean squared error loss
        
        # Store each episode in memory
        def remember(self, state, action, reward, next_state, done):
            state = torch.tensor(state).float()
            next_state = torch.tensor(next_state).float()
            self.memory.append((state, action, reward, next_state, done))
        
        # Select an action to make
        def action(self, state, model):
            # If generated number is between 0-1 is less than epsilon, select random action
            if np.random.rand() <= self.epsilon: 
                random_action = np.random.randint(0, len(self.action_space)-1) # Generate random no. between 0 - 14
                return self.action_space[random_action]
            # Otherwise select action based on neural network
            else:
                state = torch.tensor(state).float()
                action = model(state)
                return torch.argmax(action).tolist()
            
        # Train the neural network with the stored memory 
        def train(self, model):
            # Only perform training if memory is greater than the batch size
            if len(self.memory) > self.batch_size:
                batch = random.sample(self.memory, self.batch_size) # Extract a random batch of replays in memory
                for state, action, reward, next_state, done in batch:
                    current_q_values = model(state)
                    target_q_values = model(next_state)
                    if not done:
                        action_value = (reward + self.gamma * torch.max(target_q_values).tolist()) # Calculate temporal difference
                    else:
                        action_value = reward
                    target_q_values[action] = action_value

                    loss = self.mse_loss(current_q_values, target_q_values) 
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()
                    
        # Decay the exploration rate 
        def update_epsilon(self):
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay
        
        

        
        

In [ ]:
action_space = [
            (-1, 1, 0), (-1, 1, 0.4), (-1, 0, 0), (-1, 0, 0.4), (-1, 0, 1), # Action Space: (Steer, Accelerate, Brake)
            (1, 1, 0),  (1, 1, 0.4),  (1, 0, 0),  (1, 0, 0.4),  (1, 0, 1),  # Steer left = -1, Steer right = 1 
            (0, 1, 0),  (0, 1, 0.4),  (0, 0, 1),  (0, 0, 0.4),  (0, 0, 0)   # Accelerate = Between 0 - 1      
        ]       

In [ ]:
model = NN_Model()
agent = DQNAgent()


In [ ]:
for episodes in range(500): # Train for 200 epochs
    state = environment.reset() # Reset the game 
    state = torch.tensor(state).float() # Convert from numpy to tensor

    for time in range(2000):
        action = agent.action(state, model) # Choose action
        next_state, reward, done, _ = environment.step(action) # Apply the action onto the cartpole
        reward = reward if not done else -10
        agent.remember(state, action, reward, next_state, done) # Add episode to memory
        agent.train(model) #  Train agent
        agent.update_epsilon() 
        state = next_state
        if done:
            print("Episode:", episodes, "/1000", "Score:", time)
            break

        environment.render() # Display the cartpole game
environment.close() # Close the window 